In [59]:
import pandas as pd
import csv
import re
import sys
!{sys.executable} -m pip install gensim
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer

  Using cached https://files.pythonhosted.org/packages/52/d8/1a966940585bdd828d6ca8bca37d1be81e3e7e2fa1f51098117f15c32a1b/gensim-3.6.0-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/0b/3b/925c7aded1c54c12d5be964ab543420b01d95759c5756a574588d49444ad/boto3-1.9.71-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/50/c0/cd4f8bec8a10876f0ce34f0cf264fda04e09df41d1a473a43f890c71fffa/botocore-1.12.71-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/14/2a0004d487464d120c9fb85313a75cd3d71a7506955be458eebfe19a6b1d/s3transfer-0.1.13-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b7/31/05c8d001f7f87f0f07289a5fc0fc3832e9a57f2dbd4d3b0fee70e0d51365/jmespath-0.9.3-py2.py3-none-any.whl


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
data = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [5]:
bs = BeautifulSoup(data["review"][0], "lxml")

In [6]:
letters_only = re.sub("[^a-zA-Z-']", 
                      " ",
                      bs.get_text())

In [7]:
lowercase = letters_only.lower()

In [8]:
words = lowercase.split()

In [9]:
def clean(raw):
    data = BeautifulSoup(raw, "lxml").get_text() 
    letters_only = re.sub("[^a-zA-Z-']", " ", data) 
    
    words = letters_only.lower().split()
    return( " ".join( words )) 

In [10]:
Cleaner = pd.DataFrame(data)

In [11]:
data["review"] = data["review"].apply(clean)

In [13]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = "english", max_features = 5000)

In [14]:
train_data_features = vectorizer.fit_transform(data["review"])
train_data_features = train_data_features.toarray()

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
forest = RandomForestClassifier(n_estimators = 100)

In [18]:
forest = forest.fit( train_data_features, data["sentiment"] )

In [81]:
test = pd.read_csv("testData.tsv", header=0, delimiter = "\t", quoting = 3)
result = []
for review in test["review"]:
    result.append(forest.predict(vectorizer.transform([clean(review)]).toarray())[0])
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

In [42]:
data = pd.read_csv("labeledTrainData.tsv", header=0, delimiter = "\t", quoting=3)
udata = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter = "\t", quoting=3)
test = pd.read_csv("testData.tsv", header=0, delimiter = "\t", quoting = 3)

print ("read " + str(data["review"].size) + "labeled train reviews, " + str(test["review"].size) + "test reviews, and ", str(udata["review"].size) + " unlabeled reviews.")

read 25000labeled train reviews, 25000test reviews, and  50000 unlabeled reviews.


In [20]:
import nltk.data
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\evan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [43]:
tokenizer = nltk.data.load('tokenizers\punkt\english.pickle')
def sentences (review, tokenizer):
    raw_sentences = tokenizer.tokenize(review)
    i=0
    for unclean_sentence in raw_sentences:
        raw_sentences[i] = clean(raw_sentences[i])
        i = i + 1
    sentences = [] 
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentence = raw_sentence.split()
            sentences.append(sentence)
    return sentences

In [50]:
sentenceList = []
for review in data["review"]:
    sentenceList += sentences(review, tokenizer)

E:\Anaconda\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
E:\Anaconda\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
E:\Anaconda\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [51]:

for review in udata["review"]:
    sentenceList += sentences(review, tokenizer)
    

E:\Anaconda\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
E:\Anaconda\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
E:\Anaconda\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
E:\Anaconda\lib\site-packages\bs4\__init__.py:219: UserWarning

In [60]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentenceList, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

E:\Anaconda\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-01-01 21:07:55,750 : INFO : 'pattern' package not found; tag filters are not available for English
2019-01-01 21:07:55,760 : INFO : collecting all words and their counts
2019-01-01 21:07:55,761 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-01-01 21:07:55,813 : INFO : PROGRESS: at sentence #10000, processed 220735 words, keeping 19851 word types
2019-01-01 21:07:55,867 : INFO : PROGRESS: at sentence #20000, processed 442224 words, keeping 28867 word types
2019-01-01 21:07:55,917 : INFO : PROGRESS: at sentence #30000, processed 657642 words, keeping 35592 word types


Training model...


2019-01-01 21:07:55,972 : INFO : PROGRESS: at sentence #40000, processed 879534 words, keeping 41421 word types
2019-01-01 21:07:56,028 : INFO : PROGRESS: at sentence #50000, processed 1099840 words, keeping 46418 word types
2019-01-01 21:07:56,076 : INFO : PROGRESS: at sentence #60000, processed 1316019 words, keeping 50725 word types
2019-01-01 21:07:56,122 : INFO : PROGRESS: at sentence #70000, processed 1536098 words, keeping 54692 word types
2019-01-01 21:07:56,171 : INFO : PROGRESS: at sentence #80000, processed 1749981 words, keeping 58290 word types
2019-01-01 21:07:56,224 : INFO : PROGRESS: at sentence #90000, processed 1968271 words, keeping 62087 word types
2019-01-01 21:07:56,279 : INFO : PROGRESS: at sentence #100000, processed 2185268 words, keeping 65279 word types
2019-01-01 21:07:56,335 : INFO : PROGRESS: at sentence #110000, processed 2401784 words, keeping 68422 word types
2019-01-01 21:07:56,391 : INFO : PROGRESS: at sentence #120000, processed 2620140 words, keepin

2019-01-01 21:07:59,726 : INFO : PROGRESS: at sentence #760000, processed 16665913 words, keeping 194126 word types
2019-01-01 21:07:59,780 : INFO : PROGRESS: at sentence #770000, processed 16889304 words, keeping 195567 word types
2019-01-01 21:07:59,833 : INFO : PROGRESS: at sentence #780000, processed 17113405 words, keeping 196965 word types
2019-01-01 21:07:59,888 : INFO : PROGRESS: at sentence #790000, processed 17334420 words, keeping 198409 word types
2019-01-01 21:07:59,895 : INFO : collected 198569 word types from a corpus of 17359734 raw words and 791168 sentences
2019-01-01 21:07:59,896 : INFO : Loading a fresh vocabulary
2019-01-01 21:08:00,000 : INFO : effective_min_count=40 retains 16710 unique words (8% of original 198569, drops 181859)
2019-01-01 21:08:00,000 : INFO : effective_min_count=40 leaves 16638542 word corpus (95% of original 17359734, drops 721192)
2019-01-01 21:08:00,064 : INFO : deleting the raw counts dictionary of 198569 items
2019-01-01 21:08:00,071 : IN

2019-01-01 21:08:49,668 : INFO : EPOCH 4 - PROGRESS: at 62.87% examples, 860289 words/s, in_qsize 8, out_qsize 1
2019-01-01 21:08:50,687 : INFO : EPOCH 4 - PROGRESS: at 69.07% examples, 849682 words/s, in_qsize 6, out_qsize 1
2019-01-01 21:08:51,689 : INFO : EPOCH 4 - PROGRESS: at 76.04% examples, 850601 words/s, in_qsize 7, out_qsize 0
2019-01-01 21:08:52,689 : INFO : EPOCH 4 - PROGRESS: at 83.18% examples, 853270 words/s, in_qsize 7, out_qsize 0
2019-01-01 21:08:53,703 : INFO : EPOCH 4 - PROGRESS: at 89.98% examples, 852021 words/s, in_qsize 8, out_qsize 2
2019-01-01 21:08:54,714 : INFO : EPOCH 4 - PROGRESS: at 96.53% examples, 848076 words/s, in_qsize 7, out_qsize 0
2019-01-01 21:08:55,219 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-01-01 21:08:55,237 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-01-01 21:08:55,242 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-01-01 21:08:55,243 : INFO : worker thread fi

In [73]:
model.doesnt_match("good great jocund enjoyable".split())

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
2019-01-01 21:12:14,743 : WARNING : vectors for words {'jocund'} are not present in the model, ignoring these words
E:\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'enjoyable'

In [ ]:
user = input("Sentence: ")

In [85]:
print(forest.predict(vectorizer.transform([user]).toarray())[0])

0
